In [2]:
import os
import sys
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
keys_path = os.path.abspath('../../keys')
sys.path.append(keys_path)
from keys import LANGCHAIN_API_KEY, OPENAI_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

pdfs_path = os.path.abspath('../../public_data_pdfs')

In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Get all PDF files in the directory
pdf_files = [file for file in os.listdir(pdfs_path) if file.endswith('.pdf')]

# Create a list of full paths by joining the base folder with each file name
pdf_paths = [os.path.join(pdfs_path, pdf_file) for pdf_file in pdf_files]

# Initialize an empty list to hold all document splits
all_doc_splits = []

# Define the text splitter with a chunk size of 1000 tokens and 200 overlap
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, 
    chunk_overlap=200)

# Iterate over each PDF file path
for pdf_path in pdf_paths:
    # Load the PDF
    pdf_loader = PyPDFLoader(pdf_path)
    documents = pdf_loader.load()
    
    # Split the documents into chunks
    doc_splits = text_splitter.split_documents(documents)
    
    # Add the splits to the overall list
    all_doc_splits.extend(doc_splits)

In [4]:
all_doc_splits

[Document(metadata={'source': 'd:\\My Notebooks\\Master_thesis\\public_data_pdfs\\general_info1.pdf', 'page': 0}, page_content='1 General Degree Program and Examination Regulations for  \nBachelor’s and Master’s Degree Programs at the  \nFaculty of Sciences of  \nFriedrich -Alexander -Universität Erlangen -Nürnberg (FAU)  \n– ABMPO/NatFak – \nDated August 22, 2023  \nBased on Section 9 (1)(2) in conjunction with the currently valid versions of Section 80 \n(1)(2), section 84 (2)(1), Section 86 (3)(4), Section 88 (9), Section 90 (1)(2) and Section \n96 (3)(1) Bavarian Higher Education Innovation Act dated August 5, 2022 ( BayHIG ), \nFriedrich -Alexander -Universität Erlangen -Nürnberg enacts the following degree \nprogram and examination regulations:  \nContents:  \nPart I: General provisions  ................................ ................................ ................................ .....................  2 \nSection 1 Scope, Purpose of the Bachelor’s and Master’s Examination  

In [5]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import SKLearnVectorStore

vectorstore = SKLearnVectorStore.from_documents(documents=all_doc_splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [6]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)


from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [7]:
questions = [
    'Will the language of instruction of my Bachelor’s degree suffice to demonstrate my proficiency in English for admission in Msc Data Science at FAU Erlangen?',
    'Is an APS certificate mandatory for Indian students during the application process?',
    'How long does it typically take to receive a decision regarding my application to the MSc Data Science program?',
    'Am I eligible to apply for the study course program with a degree from a technical university or a university of applied sciences?',
    'Can I apply for the study course program with a degree obtained through a dual study program?',
    'What could be the reasons for receiving a rejection for my application?',
    'Is it possible to defer my admission to a future semester?',
    'Where can I find accommodation while studying at FAU?',
    'Can I switch my major subject after being admitted?',
    'Is it allowed to combine modules from different application subjects?',
    'How can I transfer ECTS credits from one module group to another?',
    'Can modules from my major subject count towards my minor subject requirements?',
    'Can my werkstudent experience be used to earn ECTS as a technical qualification?',
    'Can I participate in an examination without prior registration?',
    'How do I register for a module’s examination in Campo within the correct specialisation area?',
    'If I fail an examination on the first attempt, is it mandatory to attend the next attempt?',
    'Is it possible to withdraw from an examination after registering?',
    'What should I do if I cannot attend an examination due to illness?',
    'What are the consequences of failing an examination three times?',
    'Can I retake a completed module to improve my grades?',
    'Is it possible to improve my grades by completing additional modules?',
    'How many ECTS credits are required before I can start writing my Master’s thesis?',
    'How can I find a topic for my Master’s thesis?',
    'Who is eligible to supervise my Master’s thesis?',
    'Can my Master’s thesis supervisor be from a different department?',
    'Is it possible to complete my Master’s thesis while working in a company?',
    'Is it necessary to have a university supervisor for a company-based thesis?',
    'What is the process for registering my Master’s thesis?',
    'What steps should I take after completing my Master’s thesis?'
]

In [8]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

In [9]:
from operator import itemgetter
llm = ChatOpenAI(temperature=0)
answers = []
for question in questions:
    retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
    docs = retrieval_chain_rag_fusion.invoke({"question": question})

    from langchain_core.runnables import RunnablePassthrough

    # RAG
    template = """Answer the following question based on this context:

    {context}

    Question: {question}
    """

    prompt = ChatPromptTemplate.from_template(template)

    final_rag_chain = (
        {"context": retrieval_chain_rag_fusion, 
        "question": itemgetter("question")} 
        | prompt
        | llm
        | StrOutputParser()
    )

    ans = final_rag_chain.invoke({"question":question})
    answers.append(ans)

c:\Users\Ekaansh_Khosla\anaconda3\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [10]:
answers

['Yes, if your Bachelor’s degree was taught in English, you can apply with a certificate from your university stating that the language of instruction for your degree was in English.',
 'Yes, an APS certificate is mandatory for Indian students during the application process at FAU as well as for applying for the study visa.',
 "Answer: Typically, it takes about 4 weeks after the application deadline to receive a decision regarding your application to the MSc Data Science program. If you have not received any answer after 8 weeks, it is recommended to politely ask the Master's Office about the current state of your application.",
 'Yes, you are eligible to apply for the study course program with a degree from a technical university or a university of applied sciences. These degrees will be treated equally with a degree from a traditional university.',
 'Yes, you can apply for the study course program with a degree obtained through a dual study program. Your degree from a dual study prog